In [1]:
import sys
import importlib
import datetime

import numpy as np
import pandas as pd
import xarray as xr
import pvlib as pv
import matplotlib.pyplot as plt
import scipy.interpolate as interpolate
from scipy.ndimage.filters import uniform_filter

In [2]:
%matplotlib inline

In [3]:
sys.path.append('/home/travis/python_code/letkf_forecasting/')
import letkf_forecasting as lf
import prepare_sat_data as prep

In [4]:
sensor_CI = pd.read_hdf('/home/travis/python_code/letkf_forecasting/data/sensor_CI.h5')
sensor_loc = pd.read_hdf('/home/travis/python_code/forecasting/current_data/sensor_loc.h5')
domain_shape = np.load('./data/domain_shape.npy')
lats = np.load('./data/lats.npy')
lons = np.load('./data/lons.npy')
x = np.load('./data/x.npy')
y = np.load('./data/y.npy')
cloudiness_index = pd.read_hdf('./data/cloudiness_index.h5')
clearsky = pd.read_hdf('./data/clearsky.h5')
pixel = pd.read_hdf('./data/pixel.h5')

In [ ]:
# For 5/29
file = './data/for{suffix}/raw_winds/{variable}{suffix}.h5'
suffix = '_5_29'
U = pd.read_hdf(file.format(variable='U', suffix=suffix))
V = pd.read_hdf(file.format(variable='V', suffix=suffix))
U_shape = np.load(file.format(variable='U_shape', suffix=suffix)[:-3]+'.npy')
V_shape = np.load(file.format(variable='V_shape', suffix=suffix)[:-3]+'.npy')
wind_lats = np.load(file.format(variable='wind_lats', suffix=suffix)[:-3]+'.npy')
wind_lons = np.load(file.format(variable='wind_lons', suffix=suffix)[:-3]+'.npy')
wind_times = U.index

In [ ]:
# for 5/29
cloudiness_5_29 = cloudiness_index.ix['2014-05-29']
cloudiness_6_1 = cloudiness_index.ix['2014-06-01']
cloudiness_6_1.index = cloudiness_6_1.index - pd.Timedelta('3day')

cc_5_29 = (cloudiness_5_29 - cloudiness_6_1).clip(lower=0)

clearsky_5_29 = clearsky.ix['2014-05-29']
cloudiness_6_1 = cloudiness_index.ix['2014-06-01']
clearsky_6_1 = clearsky.ix['2014-06-01']

ktm_5_29 = (2.36*cloudiness_5_29**5 - 6.2*cloudiness_5_29**4 +
       6.22*cloudiness_5_29**3 - 2.63*cloudiness_5_29**2 -
       0.58*cloudiness_5_29 + 1)

ktm_5_29_cc = (2.36*cc_5_29**5 - 6.2*cc_5_29**4 +
             6.22*cc_5_29**3 - 2.63*cc_5_29**2 -
             0.58*cc_5_29 + 1)
# GHI_15 = ktm*clearsky_15*(0.0001*ktm*clearsky_15 + 0.9)
GHI_5_29 = ktm_5_29*(0.0001*ktm_5_29 + 0.9)

ktm_6_1 = (2.36*cloudiness_6_1**5 - 6.2*cloudiness_6_1**4 +
       6.22*cloudiness_6_1**3 - 2.63*cloudiness_6_1**2 -
       0.58*cloudiness_6_1 + 1)
# GHI_14 = ktm*clearsky_14*(0.0001*ktm*clearsky_14 + 0.9)
GHI_6_1 = ktm_6_1*(0.0001*ktm_6_1 + 0.9)
GHI_6_1.index = GHI_6_1.index - pd.Timedelta('3day')
ktm_6_1.index = ktm_6_1.index - pd.Timedelta('3day')
cloudiness_6_1.index = cloudiness_6_1.index - pd.Timedelta('3day')
# csi_15 = GHI_15/GHI_14
csi_5_29 = ktm_5_29/ktm_6_1

In [ ]:
csi_5_29 = csi_5_29.dropna()

In [ ]:
# date_range = pd.date_range('2014-04-15')
for d in csi_5_29.index[10:20]:
#     this_time = '2014-05-29 12:45:00'
    this_time = d
#     cmap = 'jet'
    cmap = 'Blues'

    # vmin=0
    # vmax=1.2

    plt.figure()
    im = plt.pcolormesh(x.reshape(domain_shape), y.reshape(domain_shape),
        (csi_5_29).ix[this_time].values.reshape(domain_shape),
        cmap=cmap)
    plt.colorbar(im)
    plt.title('csi: ' + str(this_time))
    plt.axis('equal')

In [ ]:
dx = .25
dy = .25
x_fine = np.arange(x[0], x[-1] + dx, dx)
y_fine = np.arange(y[0], y[-1] + dy, dy)
x_fine, y_fine = np.meshgrid(x_fine, y_fine)
fine_shape = x_fine.shape
x_fine = x_fine.ravel()
y_fine = y_fine.ravel()
interp_positions = np.stack([x_fine, y_fine], axis=1)
sat_times = csi_5_29.index

In [ ]:
data = csi_5_29.ix[0].values
data_positions = np.stack([x, y], axis=1)
f = interpolate.NearestNDInterpolator(data_positions, data)
data = f(interp_positions)
data = data.reshape(fine_shape)
data = uniform_filter(data, size=(4, 4)).ravel()
csi_fine = pd.DataFrame(data=data[None, :], index=[sat_times[0]])

In [ ]:
for t in np.arange(sat_times.size - 1) + 1:
    this_time = sat_times[t]
    data = csi_5_29.ix[this_time].values
    f = interpolate.NearestNDInterpolator(data_positions, data)
    data = f(interp_positions)
    data = data.reshape(fine_shape)
    data = uniform_filter(data, size=(4, 4)).ravel()
    temp = pd.DataFrame(data=data[None, :], index=[this_time])
    csi_fine = csi_fine.append(temp)

In [ ]:
csi_fine.to_hdf('./data/for_5_29/csi_5_29.h5', 'csi_5_29')

In [ ]:
data = cloudiness_5_29.values.T
data_positions = np.stack([x, y], axis=1)
f = interpolate.NearestNDInterpolator(data_positions, data)
data = f(interp_positions).T
data = data.reshape(
    [sat_times.size, fine_shape[0], fine_shape[1]])
data = uniform_filter(
    data, size=(0, 4, 4)).reshape(
    [sat_times.size, fine_shape[0]*fine_shape[1]])
cloudiness_5_29 = pd.DataFrame(data=data, index=sat_times)

In [ ]:
cloudiness_5_29.to_hdf('./data/for_5_29/cloudiness_5_29.h5', 'cloudiness_5_29')

In [ ]:
cloudiness_6_1 = cloudiness_6_1.ix[cloudiness_5_29.index]

In [ ]:
data = cloudiness_6_1.values.T
data_positions = np.stack([x, y], axis=1)
f = interpolate.NearestNDInterpolator(data_positions, data)
data = f(interp_positions).T
data = data.reshape(
    [sat_times.size, fine_shape[0], fine_shape[1]])
data = uniform_filter(
    data, size=(0, 4, 4)).reshape(
    [sat_times.size, fine_shape[0]*fine_shape[1]])
cloudiness_6_1 = pd.DataFrame(data=data, index=sat_times)

In [ ]:
cloudiness_6_1.to_hdf('./data/for_5_29/cloudiness_6_1.h5', 'cloudiness_6_1')

In [ ]:
# date_range = pd.date_range('2014-04-15')
for d in csi_5_29.index[10:11]:
#     this_time = '2014-05-29 12:45:00'
    this_time = d
#     cmap = 'jet'
    cmap = 'Blues'

    # vmin=0
    # vmax=1.2

    plt.figure()
    im = plt.pcolormesh(x_fine.reshape(fine_shape), y_fine.reshape(fine_shape),
        (csi_fine).ix[this_time].values.reshape(fine_shape),
        cmap=cmap)
    plt.colorbar(im)
    plt.title('csi: ' + str(this_time))
    plt.axis('equal')
    
    plt.figure()
    im = plt.pcolormesh(x_fine.reshape(fine_shape), y_fine.reshape(fine_shape),
        (cloudiness_5_29).ix[this_time].values.reshape(fine_shape),
        cmap=cmap)
    plt.colorbar(im)
    plt.title('ci 5/29: ' + str(this_time))
    plt.axis('equal')
    
    plt.figure()
    im = plt.pcolormesh(x_fine.reshape(fine_shape), y_fine.reshape(fine_shape),
        (cloudiness_6_1).ix[this_time].values.reshape(fine_shape),
        cmap=cmap)
    plt.colorbar(im)
    plt.title('ci 6/1: ' + str(this_time))
    plt.axis('equal')

In [ ]:
wind_x, wind_y = prep.sphere_to_lcc(wind_lats, wind_lons)

In [ ]:
wind_x = wind_x.reshape([U_shape[0], V_shape[1]])
wind_y = wind_y.reshape([U_shape[0], V_shape[1]])

In [ ]:
y_step_sn = np.diff(wind_y, axis=0).mean(axis=0)
y_step_we = np.diff(wind_y, axis=1).mean(axis=1)
x_step_sn = np.diff(wind_x, axis=0).mean(axis=0)
x_step_we = np.diff(wind_x, axis=1).mean(axis=1)

U_y = np.concatenate([wind_y - y_step_we[:, None]/2,
                      (wind_y[:, -1] + y_step_we/2)[:, None]], axis=1)
U_x = np.concatenate([wind_x - x_step_we[:, None]/2,
                      (wind_x[:, -1] + x_step_we/2)[:, None]], axis=1)

V_y = np.concatenate([wind_y - y_step_sn[None, :]/2,
                      (wind_y[-1, :] + y_step_sn/2)[None, :]], axis=0)
V_x = np.concatenate([wind_x - x_step_sn[None, :]/2,
                      (wind_x[-1, :] + x_step_sn/2)[None, :]], axis=0)
U_data_positions = np.stack([U_x.ravel(), U_y.ravel()], axis=1)
V_data_positions = np.stack([V_x.ravel(), V_y.ravel()], axis=1)

In [ ]:
dx

In [ ]:
## NEW
U_x_fine = np.concatenate([x_fine - .25/2, 
                           [x_fine[-1] + .25/2]], axis=0)
V_y_fine = np.concatenate([y_fine - dx/2,
                           [y_fine[-1] + dx/2]], axis=0)

In [ ]:
y_fine = y_fine.reshape(fine_shape)
x_fine = x_fine.reshape(fine_shape)
U_y_fine = np.concatenate([y_fine, 
                           (y_fine[:, -1])[:, None]], axis=1)
U_x_fine = np.concatenate([x_fine - .25/2, 
                           (x_fine[:, -1] + .25/2)[:, None]], axis=1)
V_y_fine = np.concatenate([y_fine - .25/2, 
                           (y_fine[-1, :] + .25/2)[None, :]], axis=0)
V_x_fine = np.concatenate([x_fine, 
                           (x_fine[-1, :])[None, :]], axis=0)
U_fine_shape = U_x_fine.shape
V_fine_shape = V_x_fine.shape
U_interp_positions = np.stack([U_x_fine.ravel(), U_y_fine.ravel()], axis=1)
V_interp_positions = np.stack([V_x_fine.ravel(), V_y_fine.ravel()], axis=1)

In [ ]:
## New
data = U.iloc[0].values.reshape(U_shape)
f = interpolate.interp2d(U_x, U_y, data, kind='linear')

In [ ]:
## New
fine_data = f(U_x_fine, y_fine)

In [ ]:
U = U.ix[wind_times]
V = V.ix[wind_times]

In [ ]:
data = U.values.T
f = interpolate.NearestNDInterpolator(U_data_positions, data)

In [ ]:
data = f(U_interp_positions).T

In [ ]:
data = data.reshape(
    [wind_times.size, U_fine_shape[0], U_fine_shape[1]])
data = uniform_filter(
    data, size=(0, 4, 4)).reshape(
    [wind_times.size, U_fine_shape[0]*U_fine_shape[1]])
U_fine = pd.DataFrame(data=data, index=wind_times)

In [ ]:
data = V.values.T
f = interpolate.NearestNDInterpolator(V_data_positions, data)
data = f(V_interp_positions).T
data = data.reshape(
    [wind_times.size, V_fine_shape[0], V_fine_shape[1]])
data = uniform_filter(
    data, size=(0, 4, 4)).reshape(
    [wind_times.size, V_fine_shape[0]*V_fine_shape[1]])
V_fine = pd.DataFrame(data=data, index=wind_times)

In [ ]:
plt.figure()
im = plt.pcolormesh(U_fine.ix['2014-05-29 12:00:00'].reshape(U_fine_shape))
plt.colorbar(im)

plt.figure()
im = plt.pcolormesh(V_fine.ix['2014-05-29 12:00:00'].reshape(V_fine_shape))
plt.colorbar(im)

In [ ]:
U_fine.to_hdf('./data/for_5_29/winds/U_fine.h5', 'U_fine')
V_fine.to_hdf('./data/for_5_29/V_fine.h5', 'V_fine')
np.save('./data/for_5_29/U_fine_shape', U_fine_shape)
np.save('./data/for_5_29/V_fine_shape', V_fine_shape)

In [ ]:
np.save('./data/for_5_29/x_fine', x_fine.ravel())
np.save('./data/for_5_29/y_fine', y_fine.ravel())
np.save('./data/for_5_29/fine_shape', fine_shape)

In [ ]:
file.format(variable='x', suffix='_3')[:-3]

In [7]:
# For 4/15
U = pd.read_hdf('./data/winds/U_15.h5')
V = pd.read_hdf('./data/winds/V_15.h5')
wind_lats = np.load('./data/winds/wind_lats.npy')
wind_lons = np.load('./data/winds/wind_lons.npy')
U_shape = np.load('./data/winds/U_shape.npy')
V_shape = np.load('./data/winds/V_shape.npy')
wind_times = pd.read_hdf('./data/winds/times.h5').index

In [6]:
# For 4/15
cloudiness_15 = cloudiness_index.ix['2014-04-15']
cloudiness_14 = cloudiness_index.ix['2014-04-14']
cloudiness_14.index = cloudiness_14.index + pd.Timedelta('1day')

cc_15 = (cloudiness_15 - cloudiness_14).clip(lower=0)

clearsky_15 = clearsky.ix['2014-04-15']
cloudiness_14 = cloudiness_index.ix['2014-04-14']
clearsky_14 = clearsky.ix['2014-04-14']

ktm_15 = (2.36*cloudiness_15**5 - 6.2*cloudiness_15**4 +
       6.22*cloudiness_15**3 - 2.63*cloudiness_15**2 -
       0.58*cloudiness_15 + 1)

ktm_15_cc = (2.36*cc_15**5 - 6.2*cc_15**4 +
             6.22*cc_15**3 - 2.63*cc_15**2 -
             0.58*cc_15 + 1)
# GHI_15 = ktm*clearsky_15*(0.0001*ktm*clearsky_15 + 0.9)
GHI_15 = ktm_15*(0.0001*ktm_15 + 0.9)

ktm_14 = (2.36*cloudiness_14**5 - 6.2*cloudiness_14**4 +
       6.22*cloudiness_14**3 - 2.63*cloudiness_14**2 -
       0.58*cloudiness_14 + 1)
# GHI_14 = ktm*clearsky_14*(0.0001*ktm*clearsky_14 + 0.9)
GHI_14 = ktm_14*(0.0001*ktm_14 + 0.9)
GHI_14.index = GHI_14.index + pd.Timedelta('1day')
ktm_14.index = ktm_14.index + pd.Timedelta('1day')
cloudiness_14.index = cloudiness_14.index + pd.Timedelta('1day')
# csi_15 = GHI_15/GHI_14
csi_15 = ktm_15/ktm_14

In [ ]:
# date_range = pd.date_range('2014-04-15')
this_time = '2014-04-15 13:00:00'
cmap='jet'

# vmin=0
# vmax=1.2
cloudiness_1415 = cloudiness_14.copy()
cloudiness_1415.index = cloudiness_14.index + pd.Timedelta('1day')

cloudiness_norm = cloudiness_15/cloudiness_1415

plt.figure()
im = plt.pcolormesh(x.reshape(domain_shape), y.reshape(domain_shape),
    (cc_15).ix[this_time].values.reshape(domain_shape),
    cmap=cmap)
plt.colorbar(im)
plt.title('cc: ' + str(this_time))
plt.axis('equal')

plt.figure()
im = plt.pcolormesh(x.reshape(domain_shape), y.reshape(domain_shape),
    (ktm_15_cc).ix[this_time].values.reshape(domain_shape),
    cmap=cmap)
plt.colorbar(im)
plt.title('ktm_cc: ' + str(this_time))
plt.axis('equal')

plt.figure()
im = plt.pcolormesh(x.reshape(domain_shape), y.reshape(domain_shape),
    (ktm_15).ix[this_time].values.reshape(domain_shape),
    cmap=cmap)
plt.colorbar(im)
plt.title('ktm: ' + str(this_time))
plt.axis('equal')

In [ ]:
dx = .25
dy = .25
x_fine = np.arange(x[0], x[-1] + dx, dx)
y_fine = np.arange(y[0], y[-1] + dy, dy)
x_fine, y_fine = np.meshgrid(x_fine, y_fine)
fine_shape = x_fine.shape
x_fine = x_fine.ravel()
y_fine = y_fine.ravel()
interp_positions = np.stack([x_fine, y_fine], axis=1)
sat_times = csi_15.index

In [ ]:
data = cloudiness_15.values.T
data_positions = np.stack([x, y], axis=1)
f = interpolate.NearestNDInterpolator(data_positions, data)
data = f(interp_positions).T
data = data.reshape(
    [sat_times.size, fine_shape[0], fine_shape[1]])
data = uniform_filter(
    data, size=(0, 4, 4)).reshape(
    [sat_times.size, fine_shape[0]*fine_shape[1]])
cloudiness_15 = pd.DataFrame(data=data, index=sat_times)

In [ ]:
data = cloudiness_14.values.T
data_positions = np.stack([x, y], axis=1)
f = interpolate.NearestNDInterpolator(data_positions, data)
data = f(interp_positions).T
data = data.reshape(
    [sat_times.size, fine_shape[0], fine_shape[1]])
data = uniform_filter(
    data, size=(0, 4, 4)).reshape(
    [sat_times.size, fine_shape[0]*fine_shape[1]])
cloudiness_14 = pd.DataFrame(data=data, index=sat_times)

In [ ]:
this_time = '2014-04-15 13:00:00'
cmap='jet'

# vmin=0
# vmax=1.2

plt.figure()
im = plt.pcolormesh(x_fine.reshape(fine_shape), y_fine.reshape(fine_shape),
    (cloudiness_15).ix[this_time].values.reshape(fine_shape),
    cmap=cmap)
plt.colorbar(im)
plt.title('cc: ' + str(this_time))
plt.axis('equal')

plt.figure()
im = plt.pcolormesh(x_fine.reshape(fine_shape), y_fine.reshape(fine_shape),
    (cloudiness_14).ix[this_time].values.reshape(fine_shape),
    cmap=cmap)
plt.colorbar(im)
plt.title('cc: ' + str(this_time))
plt.axis('equal')

plt.figure()
im = plt.pcolormesh(x_fine.reshape(fine_shape), y_fine.reshape(fine_shape),
    (cloudiness_15 - cloudiness_14).ix[this_time].values.clip(min=0).reshape(fine_shape),
    cmap=cmap)
plt.colorbar(im)
plt.title('cc: ' + str(this_time))
plt.axis('equal')

In [ ]:
data = csi_15.ix[0].values
data_positions = np.stack([x, y], axis=1)
f = interpolate.NearestNDInterpolator(data_positions, data)
data = f(interp_positions)
data = data.reshape(fine_shape)
data = uniform_filter(data, size=(4, 4)).ravel()
csi_fine = pd.DataFrame(data=data[None, :], index=[sat_times[0]])

In [ ]:
for t in np.arange(sat_times.size - 1) + 1:
    this_time = sat_times[t]
    data = csi_15.ix[this_time].values
    f = interpolate.NearestNDInterpolator(data_positions, data)
    data = f(interp_positions)
    data = data.reshape(fine_shape)
    data = uniform_filter(data, size=(4, 4)).ravel()
    temp = pd.DataFrame(data=data[None, :], index=[this_time])
    csi_fine = csi_fine.append(temp)

In [ ]:
csi_fine.to_hdf('./data/for_15/csi_15.h5', 'csi_15')

In [8]:
wind_x, wind_y = prep.sphere_to_lcc(wind_lats, wind_lons)

In [10]:
wind_x = wind_x.reshape([U_shape[0], V_shape[1]])
wind_y = wind_y.reshape([U_shape[0], V_shape[1]])

In [11]:
y_step_sn = np.diff(wind_y, axis=0).mean(axis=0)
y_step_we = np.diff(wind_y, axis=1).mean(axis=1)
x_step_sn = np.diff(wind_x, axis=0).mean(axis=0)
x_step_we = np.diff(wind_x, axis=1).mean(axis=1)

In [13]:
y_step_sn.mean()

1.7991511

In [14]:
y_step_we.mean()

-0.070897624

In [15]:
x_step_sn.mean()

0.070897706

In [16]:
x_step_we.mean()

1.7991515

In [ ]:
y_step_sn = np.diff(wind_y, axis=0).mean(axis=0)
y_step_we = np.diff(wind_y, axis=1).mean(axis=1)
x_step_sn = np.diff(wind_x, axis=0).mean(axis=0)
x_step_we = np.diff(wind_x, axis=1).mean(axis=1)

U_y = np.concatenate([wind_y - y_step_we[:, None]/2,
                      (wind_y[:, -1] + y_step_we/2)[:, None]], axis=1)
U_x = np.concatenate([wind_x - x_step_we[:, None]/2,
                      (wind_x[:, -1] + x_step_we/2)[:, None]], axis=1)

V_y = np.concatenate([wind_y - y_step_sn[None, :]/2,
                      (wind_y[-1, :] + y_step_sn/2)[None, :]], axis=0)
V_x = np.concatenate([wind_x - x_step_sn[None, :]/2,
                      (wind_x[-1, :] + x_step_sn/2)[None, :]], axis=0)
U_data_positions = np.stack([U_x.ravel(), U_y.ravel()], axis=1)
V_data_positions = np.stack([V_x.ravel(), V_y.ravel()], axis=1)

In [ ]:
y_fine = y_fine.reshape(fine_shape)
x_fine = x_fine.reshape(fine_shape)
U_y_fine = np.concatenate([y_fine, 
                           (y_fine[:, -1])[:, None]], axis=1)
U_x_fine = np.concatenate([x_fine - .25/2, 
                           (x_fine[:, -1] + .25/2)[:, None]], axis=1)
V_y_fine = np.concatenate([y_fine - .25/2, 
                           (y_fine[-1, :] + .25/2)[None, :]], axis=0)
V_x_fine = np.concatenate([x_fine, 
                           (x_fine[-1, :])[None, :]], axis=0)
U_fine_shape = U_x_fine.shape
V_fine_shape = V_x_fine.shape
U_interp_positions = np.stack([U_x_fine.ravel(), U_y_fine.ravel()], axis=1)
V_interp_positions = np.stack([V_x_fine.ravel(), V_y_fine.ravel()], axis=1)

In [ ]:
wind_times = wind_times[5:11]

In [ ]:
U = U.ix[wind_times]
V = V.ix[wind_times]

In [ ]:
data = U.values.T
f = interpolate.NearestNDInterpolator(U_data_positions, data)

In [ ]:
data = f(U_interp_positions).T

In [ ]:
data = data.reshape(
    [wind_times.size, U_fine_shape[0], U_fine_shape[1]])
data = uniform_filter(
    data, size=(0, 4, 4)).reshape(
    [wind_times.size, U_fine_shape[0]*U_fine_shape[1]])
U_fine = pd.DataFrame(data=data, index=wind_times)

In [ ]:
data = V.values.T
f = interpolate.NearestNDInterpolator(V_data_positions, data)
data = f(V_interp_positions).T
data = data.reshape(
    [wind_times.size, V_fine_shape[0], V_fine_shape[1]])
data = uniform_filter(
    data, size=(0, 4, 4)).reshape(
    [wind_times.size, V_fine_shape[0]*V_fine_shape[1]])
V_fine = pd.DataFrame(data=data, index=wind_times)

In [ ]:
# for this_time in wind_times:
#     data = U.ix[this_time].values
#     f = interpolate.NearestNDInterpolator(U_data_positions, data)
#     data = f(U_interp_positions)
#     data = data.reshape(U_fine_shape)
#     data = uniform_filter(data, size=(5, 5)).ravel()
#     data = pd.DataFrame(data=data[None, :], index=[this_time])
#     U_fine = U_fine.append(data)

#     data = V.ix[this_time].values
#     f = interpolate.NearestNDInterpolator(V_data_positions, data)
#     data = f(V_interp_positions)
#     data = data.reshape(V_fine_shape)
#     data = uniform_filter(data, size=(5, 5)).ravel()
#     data = pd.DataFrame(data=data[None, :], index=[this_time])
#     V_fine = V_fine.append(data)

In [ ]:
U_fine.to_hdf('./data/for_15/U_fine.h5', 'U_fine')
V_fine.to_hdf('./data/for_15/V_fine.h5', 'V_fine')
np.save('./data/for_15/U_fine_shape', U_fine_shape)
np.save('./data/for_15/V_fine_shape', V_fine_shape)

In [ ]:
np.save('./data/for_15/x_fine', x_fine.ravel())
np.save('./data/for_15/y_fine', y_fine.ravel())
np.save('./data/for_15/fine_shape', fine_shape)

In [ ]:
cloudiness_14.to_hdf('./data/for_15/cloudiness_14.h5', 'cloudiness_14')
cloudiness_15.to_hdf('./data/for_15/cloudiness_15.h5', 'cloudiness_15')